# [Chefkoch.de](http://www.chefkoch.de/)
------

## Ziel: 
### Scraping der Hauptrezeptesammlung von über 300'000 verschiedenen Rezepten (2.Teil)

In [1]:
import datetime as dt
import csv
from bs4 import BeautifulSoup
from lxml import html
from datetime import datetime
from multiprocessing import Pool
from random import uniform, choice
from time import sleep, time
import requests
import os.path
import re
import os
from tqdm import tqdm

In [2]:
DATAST_FOLDER = 'input/test/'
IMGS_FOLDER  = 'input/images/search_thumbnails/'
DFILE_NAME    = 'recipe_details_' + dt.datetime.now().strftime('%d-%m-%Y') + '.csv'
PIC_LIST      = 'pic_list_' + dt.datetime.now().strftime('%d-%m-%Y') + '.csv'

#### Diesmal ist das Ziel die Zutatenliste, die Zubereitung, die Tags und alle Bilder jedes einzelnen Rezeptes zu laden.
这次的目标是加载成分列表、准备、标签和每个单独食谱的所有图片
### Beispiel:

In [ ]:
from IPython.display import Image
PATH = "/Users/shu/Desktop/xyu_gatech/Food-Recipe-CNN/"
Image(filename = PATH + "dPO2Ot8.png", width='100%', height=140)

Dieses Rezept hat die Tags **Backen** und **Brot oder Brötchen**.
Ausserdem hat es 31 weitere Bilder die wir noch brauchen.

In [3]:
def get_list_of_recipes():
    recipe_links = []
    chef_file = DATAST_FOLDER + 'chefkoch_rezepte_27-12-2017.csv'
    with open(chef_file, 'r') as f:
        chefkoch = csv.reader(f)
        for row in chefkoch:
            try:
                recipe_links.append(row[-2])
            except: 
                continue 
    return(recipe_links)

def get_list_of_scraped_recipes():
    recipe_links = []
    recipes_file = DATAST_FOLDER + DFILE_NAME
    if os.path.isfile(recipes_file):
        with open(recipes_file, 'r') as f:
            chefkoch = csv.reader(f)
            for row in chefkoch:
                try:
                    recipe_links.append(row[0])
                except: 
                    continue
    return(recipe_links)

def list_to_scrape():
    print('AUFGERUFEN')
    l_scraped = get_list_of_scraped_recipes()
    l_all     = get_list_of_recipes()
    
    l_to_scrape = list(set(l_all) - set(l_scraped))
    print('Difference of sets:', len(l_to_scrape))
    
    return(l_to_scrape)

In [86]:
#recipe_links = list_to_scrape()
recipe_links =get_list_of_recipes()
len(recipe_links)

316768

In [74]:
def write_recipe_details(data):
    dpath = DATAST_FOLDER + DFILE_NAME
    with open(dpath, 'a', newline='') as f:
        writer = csv.writer(f)
        try:
            writer.writerow((data['link'],
                             data['ingredients'],
                             data['zubereitung'],
                             data['tags'],
                             data['n_pics']
                             #data['reviews'],
                             #data['gespeichert:'],
                             #data['Freischaltung:'],
                             #data['author_registration_date'],
                             #data['author_reviews']
                            ))
        except:
            writer.writerow('')

### Die Links zu den Bildern werden in einer Liste gespeichert und seperat runtergeladen.
图像的链接保存在列表中并单独下载

In [58]:
def write_picture_list(pics):
    dpath = DATAST_FOLDER + PIC_LIST
    with open(dpath, 'a', newline='') as f:  #, newline=''
        writer = csv.writer(f)
        try:
            writer.writerow(pics)
        except:
            writer.writerow('')

In [59]:
# def get_stats(soup):
#     stats = {}
#     # DROPPED
#     # anzahl bewertungen
#     reviews = soup.find('div', id="recipe__rating").find('span', class_ = "rating__total-votes m-r-s").text
#     #stats['reviews'] = reviews
    
#     # andere statistiken
#     if reviews != '(0)':
#         stats_table = soup.find('div', id="recipe-statistic").find_all('table')[1].find_all('tr')[1:5]
#     else:
#         stats_table = soup.find('div', id="recipe-statistic").find_all('table')[0].find_all('tr')[1:5]
        
#     for tr in stats_table:
#         stat_name  = tr.find_all('td')[0].text.strip()
#         stat_value = tr.find_all('td')a[1].text.strip().split(' ')[0]
#         stats[stat_name] = stat_value
#     return(stats)

# def get_n_pictures(html_text):
#     tree = html.fromstring(html_text)
#     # alle bilder links
#     links_pics = tree.xpath('//div')
#     links_pics = tree.xpath('//div[contains(@id, "slider")]//img[@class="slideshow-image lazyload"]/@src')
#     return links_pics

# def get_zubereitung(soup):
#     zuber = soup.find('div', id="rezept-zubereitung").text.strip()
#     zuber = zuber.replace('\n', ' ').replace('\r', '')
#     return zuber

# def get_ingredients(soup):
#     # liste von zubereitungen
#     ingredient_list = []
#     amounts_ingredients = soup.find('table', class_="incredients").find_all('tr')
    
#     for tr in amounts_ingredients:
#         am = tr.find_all('td')[0].text.strip() # amount
#         td = tr.find_all('td')[1].text.strip().split(',')[0] # ingredient
#         td = re.sub('\(.*?\)','', td)
#         ingr = am + '@' + td
#         ingredient_list.append(ingr)
        
#     return(str.join(',', ingredient_list))

# def get_tags(soup):
#     tags = []
#     tag_cloud = soup.find('ul', class_ = 'tagcloud').find_all('li')
#     for li in tag_cloud:
#         tags.append(li.find('a').text.strip())
        
#     return(str.join(',', tags))

# # DROPPED
# #def get_author_info(soup):
# #    author = {}
# #    author['author_registration_date'] = soup.find('div', class_="user-details").find('p').find('br').previous_sibling.strip()
# #    author['author_reviews'] = soup.find('div', class_="user-details").find('p').find('br').next_sibling.strip()
# #    return(author)

In [160]:
def get_stats(soup):
    stats = {}
    # DROPPED
    # anzahl bewertungen
    reviews = soup.find("div",class_="ds-box") [-14].text.strip()
    #stats['reviews'] = reviews
    
    # andere statistiken
    if reviews != '(0)':
        stats_table = soup.find('div', id="recipe-statistic").find_all('table')[1].find_all('tr')[1:5]
    else:
        stats_table = soup.find('div', id="recipe-statistic").find_all('table')[0].find_all('tr')[1:5]
        
    for tr in stats_table:
        stat_name  = tr.find_all('td')[0].text.strip()
        stat_value = tr.find_all('td')[1].text.strip().split(' ')[0]
        stats[stat_name] = stat_value
    return(stats)

def get_n_pictures(soup):
#     tree = html.fromstring(html_text)
    # alle bilder links
#     links_pics = tree.xpath(
#         '//div[contains(@class,"ds-mb-left recipe-image")] \
#         //img[@class="i-amphtmI-fill-content i-amphtmi-replaced-content"]/@src')
#     links_pics = tree.xpath('//div')
#     links_pics = tree.xpath('//div[contains(@id, "slider")]//img[@class="slideshow-image lazyload"]/@src')
    images = []
    for img in soup.findAll('img'):
        images.append(img.get('src'))
    if images[-1]=='https://img.chefkoch-cdn.de/img/default/layout/recipe-nopicture.jpg':
        return []
    return [images[-1]]

def get_zubereitung(soup):
    zuber = soup.find_all("div",class_="ds-box") [-14].text.strip()
    zuber = zuber.replace('\n', ' ').replace('\r', '')
    return zuber

def get_ingredients(soup):
    # liste von zubereitungen
    ingredient_list = []
    amounts_ingredients = soup.find('table', class_="ingredients table-header").find_all('tr')

    for tr in amounts_ingredients:
        am = tr.find_all('td')[0].text.strip().replace(" ","")  # amount
        td = tr.find_all('td')[1].text.strip().split(',')[0] # ingredient
        td = re.sub('\(.*?\)','', td)
        ingr = am + '@' + td
        ingredient_list.append(ingr)
        
    return(str.join(',', ingredient_list))

def get_tags(soup):
    tags = []
    tag_cloud = soup.find('ul', class_ = "r-nav-box r-std").find_all('li')
    for li in tag_cloud:
        tags.append(li.find('a').text.strip())
        
    return(str.join(',', tags))

# DROPPED
#def get_author_info(soup):
#    author = {}
#    author['author_registration_date'] = soup.find('div', class_="user-details").find('p').find('br').previous_sibling.strip()
#    author['author_reviews'] = soup.find('div', class_="user-details").find('p').find('br').next_sibling.strip()
#    return(author)

In [136]:
desktop_agents = ['Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36',
                 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36',
                 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36',
                 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1) AppleWebKit/602.2.14 (KHTML, like Gecko) Version/10.0.1 Safari/602.2.14',
                 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36',
                 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.98 Safari/537.36',
                 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.98 Safari/537.36',
                 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36',
                 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36',
                 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0']

def random_headers():
    return {'User-Agent': choice(desktop_agents),'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'}

In [137]:
failed_urls = []

In [149]:
def get_html(url):
    i = 5
    while i > 0:
        try:
            page = requests.get(url, headers=random_headers())
            if page.status_code != requests.codes.ok:
                page.raise_for_status()
            else:
#                 print(url)
                return page.text        
        except requests.exceptions.RequestException as e:
            print("Could not fetch " + url)
            print(e)
            # sichere url zu einer Liste von fehlgeschlagenen Links
            failed_urls.append(url)
            sleep(5)
            i = i - 1
            continue

    print("Link zum 5. Mal nicht abrufbar " + url + " ,abbrechen")
    return None

In [ ]:
def get_recipe_info(url):
    sleep_time = uniform(2, 4)
    sleep(sleep_time)
    
    html_text = get_html(url)
    try:
        soup = BeautifulSoup(html_text, 'lxml')

        # tags ingredients preparation
        ingredient_list = get_ingredients(soup)
        tags = get_tags(soup)
        prep = get_zubereitung(soup)
        
        # pics
        list_pics = get_n_pictures(soup)
        n_pics = len(list_pics)
        
        # update dictionary
#         stats = get_stats(soup)
        #author_info = get_author_info(soup)
#         print("stats",stats)
        
        # write dictionary
        data = {'link' : url,
                'ingredients' : ingredient_list,
                'zubereitung' : prep,
                'tags': tags, 
                'n_pics': n_pics}
#         data.update(stats)
        
        #data.update(author_info)
        
    except:
        data = ''
        list_pics = ''
        print('error')
    
    # write file 
    write_recipe_details(data)
    write_picture_list(list_pics)



In [164]:
# # Try to use the given csv for now
# len(recipe_links)
# for i in  tqdm(range(1,len(recipe_links))):
#     get_recipe_info(recipe_links[i])

  0%|                                                                                                                             | 7/316767 [00:32<392:36:00,  4.46s/it]

error


  0%|                                                                                                                             | 8/316767 [00:37<415:52:55,  4.73s/it]

error


  0%|                                                                                                                            | 52/316767 [03:51<371:52:16,  4.23s/it]

...

Could not fetch https://www.chefkoch.de/rezepte/3435791511830059/Chinagewuerz-Reis.html
404 Client Error: Not Found for url: https://www.chefkoch.de/rezepte/3435791511830059/Chinagewuerz-Reis.html
Could not fetch https://www.chefkoch.de/rezepte/3435791511830059/Chinagewuerz-Reis.html
404 Client Error: Not Found for url: https://www.chefkoch.de/rezepte/3435791511830059/Chinagewuerz-Reis.html
Could not fetch https://www.chefkoch.de/rezepte/3435791511830059/Chinagewuerz-Reis.html
404 Client Error: Not Found for url: https://www.chefkoch.de/rezepte/3435791511830059/Chinagewuerz-Reis.html
Could not fetch https://www.chefkoch.de/rezepte/3435791511830059/Chinagewuerz-Reis.html
404 Client Error: Not Found for url: https://www.chefkoch.de/rezepte/3435791511830059/Chinagewuerz-Reis.html
Could not fetch https://www.chefkoch.de/rezepte/3435791511830059/Chinagewuerz-Reis.html
404 Client Error: Not Found for url: https://www.chefkoch.de/rezepte/3435791511830059/Chinagewuerz-Reis.html


  0%|▏                                                                                                                         | 367/316767 [29:01<1200:55:23, 13.66s/it]

Link zum 5. Mal nicht abrufbar https://www.chefkoch.de/rezepte/3435791511830059/Chinagewuerz-Reis.html ,abbrechen
error


  0%|▏                                                                                                                          | 373/316767 [29:29<416:51:37,  4.74s/it]


KeyboardInterrupt: 

In [ ]:
# # it doesn't work in interactive notebook 
# p = Pool(15)
# with p:
#     p.map(get_recipe_info, recipe_links[0:50])

#### 15'360 Rezepte wurden pro Stunde in *recipe_details_27-12-2017.csv* heruntergeladen. Gesamte Laufzeit: 20h 36min
#### Die cvs Datei enthält nun die sehr wichtigen Daten

In [166]:
DFILE_NAME    = 'recipe_details_' + '27-12-2017.' + '.csv'
PIC_LIST      = 'pic_list_' + '27-12-2017' + '.csv'
print(DATAST_FOLDER+DFILE_NAME)
import pandas as pd
recipe_details = pd.read_csv(DATAST_FOLDER+DFILE_NAME, header=None)
recipe_details.head() # erste 5 zeilen

input/test/recipe_details_04-12-2022.csv


,0,1,2,3,4
0,https://www.chefkoch.de/rezepte/25655614015538...,"1kleine@Zucchini,1@Würstchen,1EL@Kräuterbutter...",Zunächst die Zucchini waschen und mit einem Sp...,"Rezepte finden,Kategorien,Rezepte eingeben",0
1,https://www.chefkoch.de/rezepte/31047514633901...,"250g@Mehl,3TL@Backpulver,100g@Crème fraîche,2@...",Ofen auf 180 °C vorheizen. Mehl mit Backpulver...,"Rezepte finden,Kategorien,Rezepte eingeben",1
2,https://www.chefkoch.de/rezepte/33703415012619...,"2½kg@Schweineschulter,200ml@Sonnenblumenöl,1TL...","kcal 1392 Eiweiß 70,97 g...","Rezepte finden,Kategorien,Rezepte eingeben",1
3,https://www.chefkoch.de/rezepte/26642514182443...,"250g@Weizenmehl,1TL,gestr.@Backpulver,88g@Zuck...",Den Süßstoff in 1-2 EL heißem Wasser auflösen....,"Rezepte finden,Kategorien,Rezepte eingeben",1
4,https://www.chefkoch.de/rezepte/28978314421493...,"4@Forelle,2EL@Zitronensaft,@Salz,@Pfeffer,1kle...",Die Backröhre auf 190 °C Ober-/Unterhitze vorh...,"Rezepte finden,Kategorien,Rezepte eingeben",1


# Herunterladen der Bilder
下载图像

Bis jetzt sind nur die Thumbnails heruntergeladen. Die Grösse beträgt 164x140. Zu finden sind die Thumbnails unter input/images/search_thumbnails. Beispiel:
到目前为止，只下载了缩略图。尺寸为 164x140。缩略图可以在 input/images/search_thumbnails 下找到。例子

In [200]:
from IPython.display import Image
PATH = "input/images/search_thumbnails/"
# Image(filename = PATH + "recipe-1742451283182013-0.jpg", width=360, height=240)

Chefkoch erlaubt seinen Nutzern Bilder vom Rezept hochzuladen. Oftmals ist es so, dass es mehr als ein Bild pro Rezept gibt. Um möglichst viele Daten zu haben, ladet man natürlich alle Bilder herunter. Die Links sind in der Datei *pic_list_27-12-2017.csv*

In [207]:
# import pandas as pd
# pic_list = pd.read_csv('input/test/pic_list_27-12-2017.csv', error_bad_lines=False, header=None)  #pic_list_04-12-2022,pic_list_27-12-2017
# pic_list.head()

/var/folders/mm/_4m99mz14qn619z8qsl3rhbh0000gn/T/ipykernel_43590/2936013834.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  pic_list = pd.read_csv('input/test/pic_list_27-12-2017.csv', error_bad_lines=False, header=None)  #pic_list_04-12-2022,pic_list_27-12-2017
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



,0
0,https://static.chefkoch-cdn.de/ck.de/rezepte/1...
1,https://static.chefkoch-cdn.de/ck.de/rezepte/1...
2,https://static.chefkoch-cdn.de/ck.de/rezepte/3...
3,https://static.chefkoch-cdn.de/ck.de/rezepte/3...
4,https://static.chefkoch-cdn.de/ck.de/rezepte/2...


Zuerst überprüfen wir ob die Liste leere Einträge hat, welche entfernt werden müssen.

In [172]:
# pic_list[pic_list[1].isnull()] # 2 NaN entries

In [211]:
# pic_list = pic_list.drop(pic_list.index[105484])
# pic_list = pic_list.drop(pic_list.index[140099])

In [222]:
# pic_list[pic_list[1].isnull()] # No more NaN entries
pic_list[0]

0        https://static.chefkoch-cdn.de/ck.de/rezepte/1...
1        https://static.chefkoch-cdn.de/ck.de/rezepte/1...
2        https://static.chefkoch-cdn.de/ck.de/rezepte/3...
3        https://static.chefkoch-cdn.de/ck.de/rezepte/3...
4        https://static.chefkoch-cdn.de/ck.de/rezepte/2...
                               ...                        
83486    https://static.chefkoch-cdn.de/ck.de/rezepte/2...
83487    https://static.chefkoch-cdn.de/ck.de/rezepte/2...
83488    https://static.chefkoch-cdn.de/ck.de/rezepte/2...
83489    https://static.chefkoch-cdn.de/ck.de/rezepte/3...
83490    https://static.chefkoch-cdn.de/ck.de/rezepte/4...
Name: 0, Length: 83491, dtype: object

**689'651** Bilder von **107'052** Rezepten müssen noch heruntergeladen werden.

### Bisherige Statistik: (Stand: Ende Dezember 2017)
    Chefkoch.de enthält:
        - Insgesamt 879'620 Bilder
        - 316'756 Rezepte
            - Davon enthalten 189'969 ein oder mehr Bilder
                - Davon enthalten 107'052 Rezepte mehr als 2 Bilder
            - 126'787 enthalten kein Bild
107,052 个食谱的 689,651 张图片尚未下载。
截至目前统计：（截至2017年12月底）
Chefkoch.de 包含：
    - 共有 879,620 张图片
    - 316'756 份食谱
        - 其中，189,969 幅包含一幅或多幅图像
            - 其中，107,052 个食谱包含超过 2 张图片
        - 126,787 不包含图像

In [296]:
import csv
links = []
with open('input/test/pic_list_27-12-2017.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        if len(row) > 1: links.append(row)

count=0
for ll in links:
    for l in ll:
        count+=1
print(count)

796727


In [345]:
partial_ids = []
title_ids = []
matches_partialid_piclists = {} # key: (partial_recipe_id,title) , value: pics_list, 
matches_partialid_value={} #key: title , value: partial_recipe_id,
for link in links:
    #get partial id
    try:
        ids=link[0].split('/')[6]
    except:
        ids='0'
        
    #get title
    try:
        title=link[0].split('fix-')[1].split('.')[0]
    except:
        title="0"
        
    partial_ids.append(ids) # extrakt partial id 提取部分标识
    title_ids.append(title)
    matches_partialid_piclists[(ids,title)]=link
    if title not in matches_partialid_value:
        matches_partialid_value[title]=[ids]
    else:
        matches_partialid_value[title].append(ids)
partial_ids[:11]


['174245',
 '8811',
 '32579',
 '187900',
 '58201',
 '80531',
 '81710',
 '128961',
 '95816',
 '231697',
 '7308']

In [342]:
title_ids[:11]

['johannisbeer-cupcakes-mit-karamellperlen',
 'lamm-aus-dem-ofen',
 'kartoffeln-mit-quark',
 'mangokuchen-mit-feinen-gewuerzen-nuessen-und-kokosflocken',
 'geschnetzeltes',
 'mango-creme',
 'zitronenkuchen-mit-sauerrahm',
 'wuerziges-sauerteigmischbrot',
 'aprikosen-bienenstich',
 'karotten-s-kartoffel-suppe',
 'bulgarisches-kaeseomelett']

In [346]:
matches_partialid_piclists[('174245','johannisbeer-cupcakes-mit-karamellperlen')]

['https://static.chefkoch-cdn.de/ck.de/rezepte/174/174245/465523-420x280-fix-johannisbeer-cupcakes-mit-karamellperlen.jpg',
 'https://static.chefkoch-cdn.de/ck.de/rezepte/174/174245/280352-420x280-fix-johannisbeer-cupcakes-mit-karamellperlen.jpg',
 'https://static.chefkoch-cdn.de/ck.de/rezepte/174/174245/280354-420x280-fix-johannisbeer-cupcakes-mit-karamellperlen.jpg']

In [347]:
matches_partialid_piclists[('8811','lamm-aus-dem-ofen')]

['https://static.chefkoch-cdn.de/ck.de/rezepte/8/8811/58347-420x280-fix-lamm-aus-dem-ofen.jpg',
 'https://static.chefkoch-cdn.de/ck.de/rezepte/8/8811/953837-420x280-fix-lamm-aus-dem-ofen.jpg',
 'https://static.chefkoch-cdn.de/ck.de/rezepte/8/8811/871097-420x280-fix-lamm-aus-dem-ofen.jpg',
 'https://static.chefkoch-cdn.de/ck.de/rezepte/8/8811/58348-420x280-fix-lamm-aus-dem-ofen.jpg',
 'https://static.chefkoch-cdn.de/ck.de/rezepte/8/8811/456140-420x280-fix-lamm-aus-dem-ofen.jpg',
 'https://static.chefkoch-cdn.de/ck.de/rezepte/8/8811/167510-420x280-fix-lamm-aus-dem-ofen.jpg',
 'https://static.chefkoch-cdn.de/ck.de/rezepte/8/8811/562782-420x280-fix-lamm-aus-dem-ofen.jpg',
 'https://static.chefkoch-cdn.de/ck.de/rezepte/8/8811/58346-420x280-fix-lamm-aus-dem-ofen.jpg',
 'https://static.chefkoch-cdn.de/ck.de/rezepte/8/8811/685461-420x280-fix-lamm-aus-dem-ofen.jpg',
 'https://static.chefkoch-cdn.de/ck.de/rezepte/8/8811/446714-420x280-fix-lamm-aus-dem-ofen.jpg',
 'https://static.chefkoch-cdn.de/

In [286]:
def get_list_of_recipes_id():
    recipe_links = []
    chef_file = DATAST_FOLDER + 'chefkoch_rezepte_27-12-2017.csv'
    with open(chef_file, 'r') as f:
        chefkoch = csv.reader(f)
        for row in chefkoch:
            try:
                recipe_links.append(row[-2])
            except:
                print('ERROR')
                continue 
    return(recipe_links)

all_ids = get_list_of_recipes_id()
all_ids_clean = []
for id in all_ids[1:]: # erste spalte überspringen, es ist der spalten name
    all_ids_clean.append(id[32:].lower()) # 'recipe-' extrahieren
del all_ids[:]
all_ids_clean[:10]



['3449791514066439/bratapfel-rotkohl.html',
 '3449721514056932/fruehstueckswolke-7-aus-fiefhusen.html',
 '3449701514054063/deichgrafensalat-von-der-westkueste.html',
 '3449651514047893/geschichteter-kohl.html',
 '3449681514052511/rinderlendenragout.html',
 '3449481514034645/blaukraut-mit-butter-maroni.html',
 '3449641514047091/sauerbraten-la-tick.html',
 '3449571514042406/punschtorte.html',
 '3449471514033424/gebackene-oberlaender.html',
 '3449241513953107/mcmoes-bunter-pasta-topf-allerlei.html']

In [348]:
#create a dict: 
matches_id_links = {} #key: recipe_id, value: links
for ids_title in all_ids_clean:
    [f_id,title0]=ids_title.split('/')
    for i in range(4,len(f_id)):
        if (str(f_id[0:i]),title0[:-5]) in matches_partialid_piclists.keys():
            matches_id_links[f_id]=matches_partialid_piclists[(f_id[0:i],title0[:-5])]
    
    
    

In [293]:
print(len(all_ids_clean))
print(len(partial_ids))
print(len(title_ids))
print(len(matches_id_links))
print(all_ids_clean[0])
print(partial_ids[0:100])
print(title_ids[0])

316767
107055
107055
3449791514066439/bratapfel-rotkohl.html
['174245', '8811', '32579', '187900', '58201', '80531', '81710', '128961', '95816', '231697', '7308', '133916', '277646', '284016', '49273', '88022', '14505', '30251', '207841', '87456', '135240', '116903', '102994', '199351', '166371', '91676', '219857', '87712', '81696', '80078', '167036', '165322', '172389', '109002', '87702', '144136', '309701', '51828', '258316', '234404', '151802', '196721', '116599', '12235', '69186', '91041', '41974', '154248', '66249', '106164', '48782', '26934', '175891', '193014', '112753', '11248', '219436', '64846', '83464', '174203', '183031', '214071', '96095', '178045', '27595', '230879', '123077', '188481', '197696', '254379', '307566', '173879', '275892', '139845', '182909', '12321', '166337', '234711', '65485', '148583', '7484', '219531', '93932', '102279', '131385', '54159', '272719', '92234', '240494', '27592', '227677', '296916', '279415', '143316', '167217', '221128', '79660', '95026', 

In [349]:
# %%time
import json
# matches = {} # key: recipe_id , value: pics_list
# lost = [] # sichere jede reciped_id wo es keinen match finden konnte => kein match
# for i in range(len(title_ids)-1):
#     match = [s for s in all_ids_clean if title_ids[i] in s and s.startswith(partial_ids[i])]
#     if len(match) == 0: # kein match
#         lost.append(i)
#         continue
#     r_id = match[0].split('/')[0]
#     matches[r_id] = links[i]

with open('input/test/matches.txt', 'w') as file:
    file.write(json.dumps(matches_id_links))


## Download von 630'770 Bildern

In [331]:
from datetime import datetime
from time import sleep
import urllib.request
from multiprocessing import Pool
from random import uniform
import os

In [332]:
# beispiel: recipe-2841831436245733-1.jpg
IMG_PATH = 'input/images/search_images/'


In [354]:
with open('input/test/matches.txt','r') as f:
    matches = json.load(f)
print(len(matches))


96945


In [333]:
def get_img_from_dict(i_dict):
    for idx, linn in enumerate(i_dict[1]):
        try:
            img_name = IMG_PATH + 'recipe-' + str(i_dict[0]) + '-' + str(idx+1) + '.jpg'
            urllib.request.urlretrieve(linn, img_name)
        except:
            print('Error:', i_dict[0])
    sleep_time = uniform(2., 3.) # schone den server mit anfragen
    sleep(sleep_time)

In [355]:
##not work for notebook 
# start_time = datetime.now()
# print(start_time)

# with Pool(20) as p:
#     p.map(get_img_from_dict, matches.items())

# print("--- %s seconds ---" % (datetime.now() - start_time))

Nach fast 5 Stunden sind alle Bilder unter /input/images/search_images/ zu finden. **32.36 GB**
### Beispiel:

In [ ]:
import os
from IPython.display import Image, display
PATH = "input/images/search_images/"
images = []
for i in range(1, 10):
    images.append(Image(filename = PATH + "recipe-378801124204434-"+str(i)+".jpg", width=420, height=280))
display(*images)